In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("../Datasets/LLM Evaluation Hub.csv")

In [ ]:
# Rename columns
df.columns = ['Text', 'Response_Options', 'Offensiveness_Label', 'Score']

# Cleaning
df['Score'].fillna(df['Score'].mean(), inplace=True)
df = df[df['Score'].isin([0, 1])]
df = df.drop('Response_Options', axis=1)

df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x='Offensiveness_Label', data=df)
plt.title('Distribution of Offensiveness Labels')
plt.show()

score_counts = df['Score'].value_counts()
labels = ['Score 0', 'Score 1']

plt.figure(figsize=(8, 6))
plt.pie(score_counts, labels=labels, autopct='%1.1f%%', colors=['#66b3ff','#ff9999'])
plt.title('Proportion of Scores')
plt.show()

In [ ]:
# Take a sample from all parts of the dataset
# 5 of every 100 rows

sampled_dfs = []

for i in range(0, len(df), 100):
    chunk = df.iloc[i:i+100]
    sampled_chunk = chunk.sample(n=5, random_state=26)
    sampled_dfs.append(sampled_chunk)
sample_df = pd.concat(sampled_dfs, ignore_index=True)

sample_df

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
import os
import time

api_key = os.environ["GEMINI_API_KEY"]
gemini_flash_llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=api_key, temperature=.3)
gemini_pro_llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=api_key, temperature=0)

prompt_template = """
Evaluate the following text to determine if it contains harmful content.

**INPUT TEXT:**
{user_input}

Respond with either "Yes" if the text is harmful or "No" if it is not.

**YOUR RESPONSE:**
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    template=prompt_template
)

In [ ]:
def langchain_predict(texts, llm, wait):
    responses = []
    for text in texts:
        formatted_prompt = prompt.format(user_input=text)
        response = llm(formatted_prompt)
        responses.append("Yes" if "yes" in response.lower() else "No")
        time.sleep(wait)
    return responses

In [ ]:
flash_preds = langchain_predict(sample_df['Text'], gemini_flash_llm, 4)

flash_preds

In [ ]:
pro_preds = langchain_predict(sample_df['Text'], gemini_pro_llm, 4)

pro_preds

In [ ]:
print(len(sample_df))
print(len(flash_preds))

In [ ]:
sample_df.loc[sample_df.index, 'Flash Predictions'] = [0 if pred == 'No' else 1 for pred in flash_preds]
sample_df.loc[sample_df.index, 'Pro Predictions'] = [0 if pred == 'No' else 1 for pred in pro_preds]
sample_df['Score'] = sample_df['Score'].astype(int)

In [ ]:
sample_df

In [ ]:
# Explicitly set data types to integers
sample_df['Score'] = sample_df['Score'].astype(int)
sample_df['Flash Predictions'] = sample_df['Flash Predictions'].astype(int)
sample_df['Pro Predictions'] = sample_df['Pro Predictions'].astype(int)

In [ ]:
from sklearn.metrics import classification_report

# Evaluate ChatGPT (via LangChain)
print("Gemini-1.5-flash Performance:")
print(classification_report(sample_df['Score'], sample_df['Flash Predictions']))
print("Gemini-pro Performance:")
print(classification_report(sample_df['Score'], sample_df['Pro Predictions']))